# Quantitative Trading and Analysis with Python: Course Project

Submitted by:

Jiawei Li (XXX)  
Sebastian Sydow (8316620)  
Strahinja Trenkic (XXX)  
Xiaoyi Zhou (XXX)  

Date: 30th March 2022

# Introduction

In this notebook, we created several functions, which perform the necessary (sub-)steps for the given task. This allowed us to easily reproduce results or change certain parameters. Please refer to the docstrings at the beginning of and the comments within the functions for further explanations. 

**Note:** Some of the explanations or quality checks in this notebook might seem too exhaustive or dispensable for the given task. Nevertheless, these explanations and quality checks were added as they facilitated in the process of understanding the concepts and shall serve as a future point of reference for us. Moreover, this notebook used some of the code provided during the lectures of the course *Quantitative Trading and Analysis with Python* (M.Sc.) at the Frankfurt School of Finance and Management.

# Preliminaries

In [ ]:
import datetime
import time
import warnings
from pathlib import Path

from matplotlib.axis import Axis
from matplotlib.pyplot import figure
from pandas.tseries.offsets import BDay
from scipy import stats
from scipy.optimize import minimize
from scipy.stats.mstats import winsorize
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from statsmodels.regression.rolling import RollingOLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
from ta.momentum import KAMAIndicator
from ta.momentum import PercentagePriceOscillator
from ta.momentum import ROCIndicator
from ta.momentum import RSIIndicator
from ta.trend import AroonIndicator
from ta.trend import EMAIndicator
from ta.trend import MACD
from tqdm import tqdm
from zipfile import ZipFile
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as web
import quandl as quandl
import quantstats as qs
import seaborn as sns
import statsmodels.api as sm
import ta as ta
import wrds as wrds

warnings.filterwarnings("ignore")

# Data Preperations

In the next cell, the data of different data sources is loaded and preprocessed. The data for the generalized lower bounds for the expected excess simple returns (`df_glb`) and  risk-neutral skewness (`df_mfis`) were provided by [Grigory Vilkov](https://www.vilkov.net/index.html) and retrieved from [here](https://osf.io/z2486/).

In [ ]:
# WRDS username
wrds_username = "iewaij"
# define the path to data folder
path = Path("./data")
# stocks
input_stocks = path / "permno_selection.csv"
# link betweeen CRSP (permno) and Option Metrics (SECID)
link_permno_secid = path / "daily_permno_secid_cusip_link.csv.zip"
# glb
glb = path / "glb_daily.csv"
# model-free implied skewness (MFIS)
mfis = path / "MFIS_1996_2019.csv"
# Save a hdf locally for easier modeling
hdf_path = path / "factors.h5"
# quandl key
quandl_key = "XWVLTpJSLyiA8s66Fh8x"
# import stock universe
df_input_stocks = pd.read_csv(input_stocks)
# glb
df_glb = pd.read_csv(glb)
# rename column
df_glb = df_glb.rename(columns={"id": "permno"})
# transfrom data type
df_glb["date"] = pd.to_datetime(df_glb["date"])
# mfis
df_mfis = pd.read_csv(mfis)
# rename column
df_mfis = df_mfis.rename(columns={"id": "permno"})
# transfrom data type
df_mfis["date"] = pd.to_datetime(df_mfis["date"])
# instantiate zip-file
zip_file = ZipFile(link_permno_secid)
# load linking table
df_link_permno_secid = pd.read_csv(zip_file.open("daily_permno_secid_cusip_link.csv"))

## Stock Returns and Market Capitalization
In the next cell, we created a function to retrieve the daily stock returns, daily stock prices and daily shares outstanding for the companies in `df_input_stocks`.

In [ ]:
def retrieve_stock_data(df_input, date_start, date_end, wrds_username):
    """
    This function retrieves the permno, cusip return, price, shares outstanding and market value of each stock
    on each day.
    
    Parameters
    ----------
    df_input: dataframe
        The dataframe storing the information about the stocks.
        
    date_start: str
        First date of data retrieval.
        
    date_end: str
        Last date of data retrieval.
    
    wrds_username: str
        Username of the wrds-account used for the data retrieval.
        
    Returns
    -------
    This functions returns the information for each stock on each day 
    as well as the weightage of each stock on each day by market value in a dataframe. 
    """
    
    # Download stock data
    ## establish WRDS connection
    db = wrds.Connection(wrds_username=wrds_username)
    # create query to load the returns, prices and shares outstanding for S&P 500 companies from 1999/01
    sql_wrds = """
            select distinct date, 
                            permno,
                            cusip, 
                            ret, 
                            abs(prc) as prc, 
                            shrout,
                            abs(prc)*shrout/1000 as mktval
            from crsp.dsf 
            where permno in %(permno)s and date>=%(start)s and date<=%(end)s
            order by date
            """

    # define the parameters
    params = {}
    params['start'] = date_start
    params['end'] = date_end
    params['permno'] = tuple(df_input.permno.unique().astype(str))

    # retrieve the data from wrds
    df_stock_data = db.raw_sql(sql_wrds, params = params)

    # change type of entries in the columns start and ending
    df_stock_data['date']  = pd.to_datetime(df_stock_data['date'])
    
    # shift market_val by 1 day
    df_stock_data.loc[:,'mktval_shifted'] = df_stock_data.groupby(['permno'])['mktval'].shift(1)
    
    # calculate weight of each stock on each day
    ## dataframe with total market value on each day
    df_ttl_mkcap = df_stock_data[['date', 'mktval_shifted']].groupby('date').sum().rename(columns={"mktval_shifted": "ttl_mktval"})
    ## merge with df_stock_data
    df_stock_data = pd.merge(df_stock_data, df_ttl_mkcap, how='left', on=['date'])
    ## calculate weightage
    df_stock_data['weightage_pct'] =  df_stock_data['mktval_shifted'] / df_stock_data["ttl_mktval"]
    ## drop columns
    df_stock_data.drop(columns = {'shrout', 'mktval', 'mktval_shifted', 'ttl_mktval'}, inplace = True) 
    db.close()
    return df_stock_data.sort_values(by=['date']).reset_index(drop=True)

## Fama-French Factors
In the next cell, we created a function to download the data for the four-factor model (daily) from the Ken French data library.

In [ ]:
def download_ffm(date_start, date_end):
    """
    This function download the data for the four-factor model (daily) from the Ken French data library.

    Parameters
    ----------
    date_start: str
        First date of data retrieval.
        
    date_end: str
        Last date of data retrieval.
    
    Returns
    -------
    This functions returns the data for the four-factor model (daily) from the Ken French data library
    in a dataframe. 
    """
    
    # start of time period 
    startdt = datetime.datetime(int(date_start[:4]),
                                int(date_start[5:7]),
                                int(date_start[8:]))
    
    # end of time period                            
    enddt = datetime.datetime(int(date_end[:4]),
                              int(date_end[5:7]),
                              int(date_end[8:]))

    # define which dataset are to be downloaded
    d1 = web.DataReader('F-F_Research_Data_Factors_daily','famafrench',start=startdt, end=enddt)
    d2 = web.DataReader('F-F_Momentum_Factor_daily','famafrench',start=startdt, end=enddt)

    # key is 0 -> get returns data
    # divide by 100 to get the returns
    df_ff_3factor = d1[0]/100

    # add momentum factor with an outer-join
    # outer-join: keep all data -> union
    df_ff_4factor = df_ff_3factor.join(d2[0]/100, how = 'outer')

    # reset index for merge later
    df_ff_4factor = df_ff_4factor.reset_index()

    # change columns to be small letters and get rid of white-spaces
    df_ff_4factor.columns = [z.lower().strip() for z in df_ff_4factor.columns]

    # rename column
    df_ff_4factor.rename(columns = {'mkt-rf':'mktrf'}, inplace = True)

    # change order of dataframe
    df_ff_4factor = df_ff_4factor.loc[:,['date', 'mktrf', 'smb', 'hml', 'mom', 'rf']]
                              
    return df_ff_4factor

## Quandl Data

In the next cell, we created a function to download data from quandl, among others:

**Crude Oil Prices**
- [WTI Spot Price](https://data.nasdaq.com/data/EIA/PET_RWTC_D-cushing-ok-wti-spot-price-fob-daily)

**Treasury Yields**
- [US Treasury Yields](https://data.nasdaq.com/data/USTREASURY/YIELD-treasury-yield-curve-rates)

**Gold**
- [LBMA/GOLD](https://data.nasdaq.com/data/LBMA/GOLD-gold-price-london-fixing)


In [ ]:
def get_quandl_data(quandl_key):
    """
    This function download the data from quandl, which can later be utilized to construct factors.

    Parameters
    ----------
    quandl_key: path
        Path to an excel-file, which stores the quandl-key in the first cell.
    
    Returns
    -------
    This functions returns three dataframes: 
    - Daily WTI Spot Prices
    - Daily yield difference between 10year yield and 1year yield of US treasuries
    - Daily price of gold
    """
    
    # configure quandl
    quandl.ApiConfig.api_key = quandl_key
    
    # Crude Oil
    df_crude_oil = quandl.get("EIA/PET_RWTC_D")
    # calculate daily returns
    df_crude_oil = df_crude_oil.pct_change(1)
    # reset_index
    df_crude_oil.reset_index(inplace=True)
    # rename column
    df_crude_oil.rename(columns={"Value": "ret_crude_oil", "Date": "date"}, inplace=True)
    
    # Treasury Yields
    df_treasury_yield = quandl.get("USTREASURY/YIELD")
    # drop columns
    df_treasury_yield.drop(labels=['1 MO', '2 MO', '3 MO', '6 MO', '2 YR', '3 YR', '5 YR', '7 YR', '20 YR', '30 YR'], axis=1, inplace=True)
    # calculate difference between 10year yield and 1year yield
    df_treasury_yield['yield_dif_10y_1y'] = df_treasury_yield['10 YR'] - df_treasury_yield['1 YR']
    # drop columns
    df_treasury_yield.drop(labels=['1 YR', '10 YR'], axis=1, inplace=True)
    # reset_index
    df_treasury_yield.reset_index(inplace=True)
    # rename column
    df_treasury_yield.rename(columns={"Date": "date"}, inplace=True)
    
    # Gold
    df_gold = quandl.get("LBMA/GOLD")
    # drop columns
    df_gold.drop(labels=['USD (PM)','GBP (AM)', 'GBP (PM)', 'EURO (AM)', 'EURO (PM)'], axis=1, inplace=True)
    # calculate daily returns
    df_gold = df_gold.pct_change(1)
    # reset_index
    df_gold.reset_index(inplace=True)
    # rename column
    df_gold.rename(columns={"USD (AM)": "ret_gold", "Date": "date"}, inplace=True)
    
    return df_crude_oil, df_treasury_yield, df_gold

## Exchange Rate

Data about exchange rates were retrieved from the St. Louis FED with the function `get_fx_data`.
- [Exchange Rates ST. Louis FRED](https://fred.stlouisfed.org/categories/15)
- [Nominal Emerging Market Economies U.S. Dollar Index](https://fred.stlouisfed.org/series/DTWEXEMEGS), only available from 2006 onwards
- [Real Emerging Market Economies Dollar Index](https://fred.stlouisfed.org/series/RTWEXEMEGS), only available from 2006 onwards

In [ ]:
def get_fx_data(symbol, start_date):
    """
    This function downloads the data about exchange rates from the ST. Louis FED.

    Parameters
    ----------
    symbol: symbol
        Identifier of the data provided by the St. Louis FED.
        
    start_date: datetime
        First date of data retrieval.
    
    Returns
    -------
    This functions returns the dataframe with the daily returns of the forex data. 
    """
    # get data
    df_fx = web.DataReader(symbol, 'fred', start_date)
    # calculate daily returns
    df_fx = df_fx.pct_change(1)
    # reset index
    df_fx.reset_index(inplace=True)
    # rename
    df_fx.rename(columns={'DATE': 'date', symbol: 'fx_rates'}, inplace=True)
    
    return df_fx

## Calculate Excess Return

In [ ]:
def calc_excess_return(df, minuend, subtrahend):
    """
    This function calculates the excess return.

    Parameters
    ----------
    df: dataframe
        Identifier of the data provided by the St. Louis FED.

    minuend: str
        Column storing data about the minuend.

    subtrahend: str
        Column storing data about the subtrahend.

    Returns
    -------
    This functions returns the dataframe with an additional column for the excess return.
    """
    # calculate excess return
    df["excess_return"] = df[minuend] - df[subtrahend]
    return df


## Perfrom Data Preparation 

In [ ]:
def data_prep(
    df_input,
    date_start,
    date_end,
    df_link,
    wrds_username,
    minuend,
    subtrahend,
    quandl_key,
):
    """
    This function calls the previously created functions to load and pre-process the data.

    Parameters
    ----------
    df_input: dataframe
        The dataframe storing the information about the stocks.

    date_start: str
        First date of data retrieval.

    date_end: str
        Last date of data retrieval.

    df_link: dataframe
        The dataframe storing the mapping between CRSP [permno] and OptionMetrics [secid] identifiers.

    wrds_username: str
        Username of the wrds-account used for the data retrieval.

    minuend: str
        Column storing data about the minuend.

    subtrahend: str
        Column storing data about the subtrahend.

    quandl_key: path
        Path to an excel-file, which stores the quandl-key in the first cell.

    Returns
    -------
    This functions returns the downloaded and pre-processed data as a dataframe.
    """
    # retrieve data
    df_stock_data = retrieve_stock_data(
        df_input=df_input,
        date_start=date_start,
        date_end=date_end,
        wrds_username=wrds_username,
    )

    # change type of entries in the columns date
    df_link["date"] = pd.to_datetime(df_link["date"])
    # merge with cusip
    df_stock_data = pd.merge(
        df_stock_data,
        df_link.loc[:, ["permno", "date", "secid"]],
        how="left",
        on=["permno", "date"],
    )
    # Download Fama-French Factors
    df_ffm = download_ffm(date_start=date_start, date_end=date_end)
    # merge stock data with ffm
    df_stocks_factors = pd.merge(df_stock_data, df_ffm, how="inner", on=["date"])
    # calculate excess return
    df_stocks_factors = calc_excess_return(
        df=df_stocks_factors, minuend=minuend, subtrahend=subtrahend
    )
    # get quandl data
    df_crude, df_yields, df_gold = get_quandl_data(quandl_key=quandl_key)
    # merge with stock data
    df_stocks_factors = pd.merge(df_stocks_factors, df_crude, how="left", on=["date"])
    df_stocks_factors = pd.merge(df_stocks_factors, df_yields, how="left", on=["date"])
    df_stocks_factors = pd.merge(df_stocks_factors, df_gold, how="left", on=["date"])

    return df_stocks_factors


In the next cell, we call the function `data_prep` to download, pre-process and merge the data.

In [ ]:
df_prepared = data_prep(
    df_input=df_input_stocks,
    date_start="1999-01-01",
    date_end="2019-12-31",
    df_link=df_link_permno_secid,
    wrds_username=wrds_username,
    minuend="ret",
    subtrahend="rf",
    quandl_key=quandl_key,
)

## Estimation of $\beta$-Factors

In [ ]:
def estimate_betas(df, window_size, factors, period="daily"):
    """
    This function calls the previously estimates the beta-factors for those characteristics,
    which are not directly observable for each stock.

    Parameters
    ----------
    df_input: dataframe
        The dataframe storing the information about the stocks.

    windo_size: int
        The window for whcich the betas shall be estimated.

    factors: list
        List of factors, for which betas need to be estimated.

    period: str
        The period of recalculating the betas, (daily or monthly).

    Returns
    -------
    This functions returns dataframe with etimated betas, which are unique for each stock.
    """
    # identify all companies (permnos) in the dataframe
    permnos = df.permno.unique()
    # loop over all permnos and perform ols regression
    for permno in tqdm(range(len(permnos))):
        # define dataframe only with current permno
        df_current_permno = df[df["permno"] == permnos[permno]]
        # only perform OLS for stocks with more than 252 observations
        if len(df_current_permno) > window_size:
            # dependent variable
            Y = df_current_permno["excess_return"]
            # independent variable
            X = df_current_permno[factors]
            # define constant
            X_constant = sm.add_constant(X)
            # define model
            rol_ols_model = RollingOLS(endog=Y, exog=X_constant, window=window_size)
            # fitting
            # print('Fitting rolling OLS model for permno #', permno, ' from', len(permnos))
            results = rol_ols_model.fit()
            # parameters
            if permno == 0:
                df_params = results.params
            else:
                df_params = pd.concat([df_params, results.params])

    # rename columns
    for col in factors:
        df_params.rename(columns={col: col + str("_beta")}, inplace=True)
    # df_params.rename(columns = {'mktrf':'beta1', 'smb':'beta2', 'hml':'beta3', 'mom': 'beta4'}, inplace = True)
    # merge with df_all by index
    print("Merging the dataframes...")
    df_betas = pd.merge(
        df,
        df_params,
        how="left",
        left_index=True,
        right_index=True,
        suffixes=[None, None],
    )
    print("Merging completed.")

    if period == "daily":
        df_betas.dropna(inplace=True)
        df_betas = df_betas.reset_index(drop=True)
        return df_betas

    if period == "monthly":
        # add '_factor' to the entries in list factors
        ls_factors_betas = [
            factors[i] + str("_beta") for i in list(range(len(factors)))
        ]
        # create a temporary copy of df_betas
        cols2copy = ["permno", "date", "const"] + ls_factors_betas
        df_temp = df_betas[cols2copy].copy()
        # df_temp = df_betas[['permno', 'date', 'const', 'beta1', 'beta2', 'beta3', 'beta4']].copy()

        # add necessary columns for group by
        df_temp["year"] = df_temp["date"].dt.year
        df_temp["month"] = df_temp["date"].dt.month
        # group dataframe and select last row
        df_temp = df_temp.groupby(["permno", "year", "month"]).tail(n=1)

        # rename columns to indicate that those are the betas of the end of each month
        for col in cols2copy[2:]:
            df_temp.rename(columns={col: col + str("_eom")}, inplace=True)
        # df_temp.rename(columns = {'const':'const_eom', 'beta1':'beta1_eom', 'beta2':'beta2_eom', 'beta3':'beta3_eom', 'beta4':'beta4_eom'}, inplace = True)
        # merge with df_betas
        df_betas_monthly = pd.merge(
            df_betas, df_temp, how="left", on=["date", "permno"]
        )
        # forward fill estimaed beta factors to assume that they stay constant for the next month until the last day
        gb = df_betas_monthly.groupby("permno")
        cols_eom = cols2copy[2:]
        cols_eom = [cols_eom[i] + str("_eom") for i in list(range(len(cols_eom)))]
        for var in cols_eom:
            df_betas_monthly[var] = gb[var].ffill()

        # drop daily betas
        # df_betas_monthly.drop(columns = {'const', 'beta1', 'beta2', 'beta3', 'beta4', 'year', 'month'}, inplace = True)
        df_betas_monthly.drop(
            ["const", "year", "month"] + ls_factors_betas + factors,
            axis=1,
            inplace=True,
        )
        # drop nas
        df_betas_monthly.dropna(inplace=True)

        # reset index of df
        df_betas_monthly = df_betas_monthly.reset_index(drop=True)

        return df_betas_monthly

In [ ]:
factors = [
    "mktrf",
    "smb",
    "hml",
    "mom",
    "ret_crude_oil",
    "yield_dif_10y_1y",
    "ret_gold",
]

df_betas = estimate_betas(
    df=df_prepared, window_size=252, factors=factors, period="monthly"
)

## Plot Benchmark Return

In [ ]:
def weighted_avg(df, values, weights):
    """
    Calculate the valued weighted average.
    """
    d = df[values]
    w = df[weights]
    return (d * w).sum() / w.sum()

In [ ]:
def plot_benchmark_performance(df, column_ret, column_weight):
    # calculate benchmark performance on each day
    df_plot = df.groupby(["date"]).apply(weighted_avg, column_ret, column_weight) + 1
    df_plot.iloc[0] = df_plot.iloc[0] * 100

    # size of the plot
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))

    # define the spacing on the x-axis
    ## major ticks every 4 years
    major_ticks_years = mdates.YearLocator(4)
    Axis.set_major_locator(ax.xaxis, major_ticks_years)
    ## Minor ticks every year
    minor_ticks_year = mdates.YearLocator(1)
    Axis.set_minor_locator(ax.xaxis, minor_ticks_year)

    # title
    plt.title(r"Value-Weighted Excess-Return of the Benchmark-Portfolio")

    # format the date type to year on the x-axis
    ax.format_xdata = mdates.DateFormatter("% m")

    # 9/11
    ax.annotate(
        "9/11",
        xy=("2001-09-11 00:00:00", 120),
        xytext=(15, 15),
        textcoords="offset points",
        arrowprops=dict(arrowstyle="-|>"),
    )
    ax.axvline(x=pd.to_datetime("2001-09-11"), color="r", linestyle="--", lw=1)

    # bankruptcy Lehman
    ax.annotate(
        "Bankruptcy Lehman Brothers",
        xy=("2008-09-15 00:00:00", 90),
        xytext=(15, 15),
        textcoords="offset points",
        arrowprops=dict(arrowstyle="-|>"),
    )
    ax.axvline(x=pd.to_datetime("2008-09-15"), color="r", linestyle="--", lw=1)

    # covid
    ax.annotate(
        "Covid-19",
        xy=("2020-03-16 00:00:00", 350),
        xytext=(-60, 5),
        textcoords="offset points",
        arrowprops=dict(arrowstyle="-|>"),
    )
    ax.axvline(x=pd.to_datetime("2020-03-16"), color="r", linestyle="--", lw=1)

    # define start and end of x-axis
    ax.set_xlim(min(df["date"]), max(df["date"]))

    # define labels of x- and y-axis
    ax.set(
        xlabel="Date",
        ylabel="Performance in % (Value-Weighted Mean of Benchmark Portfolio)",
    )

    ax.plot(df_plot.cumprod(), label="Excess Return Benchmark")
    ax.legend(loc="upper left")

In [ ]:
plot_benchmark_performance(df=df_betas, column_ret="ret", column_weight="weightage_pct")

## Option-Implied Data

### GLB and MFIS

In the next cell, the data for the generalized lower bounds for the expected excess simple returns (`df_glb`) and  risk-neutral skewness (`df_mfis`) are added.

In [ ]:
# merge data with glb
df_stocks_betas_options = pd.merge(df_betas, df_glb, how="left", on=['permno', 'date'])
# merge data with mfis
df_stocks_betas_options = pd.merge(df_stocks_betas_options, df_mfis, how='left', on=['permno', 'date'])

### Average Volatility and SKEW

In [ ]:
def retrieve_option_metrics(
    df,
    start_year,
    end_year,
    days,
    delta_otm_put,
    delta_atm_call,
    delta_otm_put2,
    delta_atm_call2,
    wrds_username,
):

    """
    This function retrieves the option metrics from WRDS.

    Parameters
    ----------
    df: dataframe
        The dataframe storing the information about the stocks.

    start_year: int
        First year of data retrieval.

    end_year: int
        Last year of data retrieval.

    days: int
        Number of days used for calulating the metrics.

    delta_otm_put: int
        Used for calculating skew, OTM put (delta)

    delta_atm_call: int
        Used for calculating skew, ATM put (delta)

    delta_otm_put2: int
        Used for calculating skew2, OTM put (delta)

    delta_atm_call2: int
        Used for calculating skew2, ATM put (delta)

    wrds_username: str
        Username of the wrds-account used for the data retrieval.

    Returns
    -------
    This functions returns the downloaded option metrics as a dataframe.
    """

    ## establish WRDS connection
    db = wrds.Connection(wrds_username=wrds_username)

    # define the parameters
    params = {}
    params["secids"] = tuple(df.secid.unique())
    params["days"] = days
    params["delta_otm_put"] = delta_otm_put
    params["delta_atm_call"] = delta_atm_call
    params["delta_otm_put2"] = delta_otm_put2
    params["delta_atm_call2"] = delta_atm_call2

    # define the sql query
    sql = """
    select a.date, a.secid, a.iv-b.iv as skew1, c.iv, (g.iv-v.iv)/a.iv as skew2 from 

    (select date, secid, impl_volatility as iv 
    from optionm.vsurfd%(year)s
    where secid in %(secids)s and days = %(days)s and delta=%(delta_atm_call)s) as a, 

    (select date, secid, impl_volatility as iv 
    from optionm.vsurfd%(year)s
    where secid in %(secids)s and days = %(days)s and delta=%(delta_otm_put)s) as b, 

    (select date, secid, AVG(impl_volatility) as iv 
    from optionm.vsurfd%(year)s
    where secid in %(secids)s and days = %(days)s and abs(delta)<=%(delta_atm_call)s
    group by date, secid) as c,
    
    (select date, secid, impl_volatility as iv 
    from optionm.vsurfd%(year)s
    where secid in %(secids)s and days = %(days)s and delta=%(delta_otm_put2)s) as g, 

    (select date, secid, impl_volatility as iv 
    from optionm.vsurfd%(year)s
    where secid in %(secids)s and days = %(days)s and delta=%(delta_atm_call2)s) as v

    where a.date= b.date and a.secid = b.secid 
    and a.date = c.date and a.secid = c.secid
    and a.date = g.date and a.secid = g.secid
    and a.date = v.date and a.secid = v.secid
    """
    # retrieve data for 1999
    params["year"] = start_year
    df_optiondata = db.raw_sql(sql, params=params)

    # append data for each year from 2000 to 2020
    for y in tqdm(range(start_year + 1, end_year)):
        # print('Processing year ', y)
        # set parameter for the year
        params["year"] = y
        df_optiondata = pd.concat([df_optiondata, db.raw_sql(sql, params=params)])
    db.close()
    return df_optiondata

In [ ]:
df_option_metrics = retrieve_option_metrics(
    df=df_stocks_betas_options,
    start_year=1999,
    end_year=2020,
    days=30,
    delta_otm_put=-10,
    delta_atm_call=50,
    delta_otm_put2=-25,
    delta_atm_call2=25,
    wrds_username=wrds_username,
)

# change type of entries in the column data
df_option_metrics.loc[:, "date"] = pd.to_datetime(df_option_metrics.loc[:, "date"])

# merge optiondata and stockdata
df_stocks_betas_options = pd.merge(
    df_stocks_betas_options, df_option_metrics, how="left", on=["secid", "date"]
)

### Variance Risk Premium

In [ ]:
def get_vrp(wrds_username):
    db = wrds.Connection(wrds_username=wrds_username)

    sql = """
    select a.date,a.secid,a.return
    from optionm.secprd%(year)s as a
    where secid in %(secids)s
    """
    params = {}
    params["secids"] = tuple(df_stocks_betas_options.secid.unique())
    params["year"] = 1999
    options_returns = db.raw_sql(sql, params=params)

    for y in tqdm(range(2000, 2020)):
        params["year"] = y
        options_returns = pd.concat([options_returns, db.raw_sql(sql, params=params)])

    options_returns.loc[:, "date"] = pd.to_datetime(options_returns.loc[:, "date"])
    options_returns = options_returns.rename(columns={"return": "option_returns"})
    options_returns = options_returns.set_index("date")

    temp1 = (
        options_returns.groupby(by=["secid"])["option_returns"].rolling(window=21).var()
        * 252
    )
    temp1.name = "realized_variance"
    temp1 = temp1.reset_index()
    db.close()

    df_stocks_betas_options = df_stocks_betas_options.merge(
        temp1, how="left", on=["date", "secid"]
    )
    df_stocks_betas_options.loc[:, "vrp"] = (
        df_stocks_betas_options.loc[:, "iv"] ** 2
    ).rolling(21).sum() / (df_stocks_betas_options.loc[:, "realized_variance"]).rolling(
        21
    ).sum()
    df_stocks_betas_options.drop(["realized_variance"], inplace=True, axis=1)

    df_stocks_betas_options.to_hdf(
        hdf_path, key="factors", mode="a", data_columns=True, complib="zlib"
    )
    return df_stocks_betas_options

In [ ]:
df_stocks_betas_options = get_vrp(wrds_username=wrds_username)

## Technical Indicators

In [ ]:
def technicals_rock(df_a):
    KAMA = KAMAIndicator(close = df_a["prc"],window = 10,pow1 = 2, pow2 = 30)
    PPO = PercentagePriceOscillator(close = df_a["prc"],window_slow = 26, window_fast = 12, window_sign = 9)
    ROCI = ROCIndicator(close=df_a["prc"],window=12)
    RSI = RSIIndicator(close=df_a["prc"],window = 14)
    EMA14 = EMAIndicator(close = df_a["prc"],window = 14)
    EMA25 = EMAIndicator(close = df_a["prc"],window = 25)
    MACD_ind = MACD(close = df_a["prc"],window_slow = 26,window_fast = 12,window_sign=9)
    Aroon = AroonIndicator(close = df_a["prc"],window = 25)
    
    df_a = df_a.assign(    
    KAMA = KAMA.kama(),
    PPO = PPO.ppo_signal(),
    ROCI = ROCI.roc(),
    RSI = RSI.rsi(),
    EMA14 = EMA14.ema_indicator(),
    EMA25 = EMA25.ema_indicator(),
    MACD = MACD_ind.macd_signal(),
    AroonInd = Aroon.aroon_indicator()
    )
    
    return df_a

In [ ]:
df_stocks_betas_options = technicals_rock(df_stocks_betas_options)

# Data Wrangling

## Delete Columns with Too Many Missing Values

In [ ]:
def count_nan(df):
    """
    This function counts None values on each column plots the count as a bar-chart.

    Parameters
    ----------
    df_input: dataframe
        The dataframe with the columns for which the count of None values shall be plotted.

    Returns
    -------
    This functions returns the plotted None values as a bar-chart.
    """
    # Count NaN
    df_count_nan = df.isna().sum().reset_index(name="Count NaN")
    # Rename column
    df_count_nan = df_count_nan.rename(columns={"index": "Column"})
    # Plot
    df_count_nan.plot.bar(
        x="Column", y="Count NaN", rot=45, figsize=(16, 6), title="Count NaN per Column"
    )

In [ ]:
count_nan(df=df_stocks_betas_options)

In [ ]:
# drop columns with many nas
df_stocks_betas_options.drop(
    [
        "glb2_D182",
        "glb3_D182",
        "glb2_D273",
        "glb3_D273",
        "glb2_D365",
        "glb3_D365",
        "mfis182",
        "mfis273",
        "mfis365",
    ],
    inplace=True,
    axis=1,
)

## Multicollinearity
Multicollinearity occurs when there are two or more independent variables in a multiple regression model, which have a high correlation among themselves (see also [here](https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/) and [here](https://www.investopedia.com/terms/m/multicollinearity.asp)). This may lead to difficulties, when trying to distinguis between their individual effects on the dependent variable. <br> 
In statistics, one commone approach to quantify a relationship between two variables is to the [Pearson correlation coefficient](https://www.statology.org/pearson-correlation-coefficient/), which is a measure of the linear association between two variables. It has a value between -1 and 1 where:
- -1 indicates a perfectly negative linear correlation between two variables
- 0 indicates no linear correlation between two variables
- 1 indicates a perfectly positive linear correlation between two variables

The further away the Pearson correlation coefficient is away from zero, the stronger the relationship between the two variables.
The pairwise Pearson correlation coefficient of the respective variables can be clearly visualized using the correlation matrix as a heatmap.

In [ ]:
def corrmatrix_heatmap(df, columns):
    """
    This function calculates the pairwise Pearson correlation coefficient of the respective variables
    and plots them in a heatmap.
    
    Parameters
    ----------
    df: dataframe
        The dataframe storing the information about the stocks.
        
    columns: list
        The variables for which the Pearson correlation coefficient shall be calculated.
        
    Returns
    -------
    This functions returns the heatmap of the pairwise Pearson correlation coefficient. 
    """
    
    X = df[columns]

    # Compute the correlation matrix
    corrMatrix = X.corr()

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(14, 11))

    sns.heatmap(corrMatrix, annot=True)

In [ ]:
# define columns to calculate pearson correlation coefficient for
idx = list(df_stocks_betas_options.columns).index("excess_return")
columns = list(df_stocks_betas_options.columns)[idx+1:]

# execute function
corrmatrix_heatmap(df=df_stocks_betas_options, columns=columns)

In the next cells, we use the [variance inflation factor (VIF)](https://www.statsmodels.org/devel/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html) to additionally measure the amount of collinearity. One recommendation is that if VIF is greater than 5, then the explanatory variable given by exog_idx is highly collinear with the other explanatory variables, and the parameter estimates will have large standard errors because of this.

In [ ]:
def VIF_Features(df):
    """
    This function calculates the variance inflation factor (VIF).

    Parameters
    ----------
    df: dataframe
        The dataframe storing the information about the stocks.

    Returns
    -------
    This functions returns variance inflation factors.
    """

    X_ = df.copy()
    X_ = X_[X_.columns[10:]]
    X_ = X_.dropna()

    vif_data = pd.DataFrame()
    vif_data["feature"] = X_.columns
    vif_data["VIF"] = [
        variance_inflation_factor(X_.values, i) for i in range(len(X_.columns))
    ]

    return vif_data

In [ ]:
# drop columns
df_stocks_betas_options.drop(
    ["glb2_D30", "glb2_D91", "glb3_D91", "mfis30", "EMA25", "EMA14", "KAMA"],
    inplace=True,
    axis=1,
)

In [ ]:
dd = VIF_Features(df_stocks_betas_options)
dd

## Split and Preprocessing 

In [ ]:
def alpha_returns(df, kind):
    if kind == "Alpha":
        benchmark_returns = df.groupby(["date"])[["weightage_pct", "ret"]].apply(
            lambda x: np.nansum(x["ret"] * x["weightage_pct"])
        )
        benchmark_returns.name = "ret_benchmark"
        df = df.merge(benchmark_returns, on=["date"])
        df["ret_bm_fw"] = df.groupby(["permno"])["ret_benchmark"].shift(-1)
        df.loc[:, "fret1d"] = df.groupby(["permno"])["ret"].shift(-1)
        df["fret1d"] = df["fret1d"] - df["ret_bm_fw"]
        df.drop(columns={"ret_bm_fw", "ret_benchmark"}, inplace=True)
    if kind == "Regular":
        df.loc[:, "fret1d"] = df.groupby(["permno"])["ret"].shift(-1)
    return df

In [ ]:
df_stocks_betas_options = alpha_returns(df_stocks_betas_options, kind="Regular")

In [ ]:
def train_test_split_sorted(df, date_of_split, dates):

    date_of_split = pd.to_datetime(date_of_split)
    X_train = df[
        (df["date"] <= date_of_split) & (df["date"] > pd.datetime(1999, 12, 31))
    ]
    X_test = df[(df["date"] > date_of_split) & (df["date"] < pd.datetime(2020, 1, 1))]

    X_train = X_train.sort_values(by=["date", "permno"])
    X_test = X_test.sort_values(by=["date", "permno"])

    return X_train, X_test

In [ ]:
def process(df, columns):
    df_scaled = df.copy()

    df_scaled = df_scaled.fillna(method="ffill", limit=30, inplace=False)
    df_scaled = df_scaled.dropna()

    for column in columns:
        df_scaled[column] = (df_scaled[column] - df_scaled[column].min())  / (df_scaled[column].max() - df_scaled[column].min())
        df_scaled[column], fitted_lambda = stats.boxcox(df_scaled[column] + 0.0001) 
        df_scaled.loc[:,column] = winsorize(df_scaled.loc[:,column], limits=0.01).data
        df_scaled[column] -= df_scaled[column].mean()
        
    return df_scaled


In [ ]:
df_stocks_betas_options.loc[:, "date"] = pd.to_datetime(
    df_stocks_betas_options.loc[:, "date"]
)

train, test = train_test_split_sorted(
    df=df_stocks_betas_options,
    date_of_split=pd.datetime(2012, 12, 31),
    dates=df_stocks_betas_options.date,
)

columns2norm = train.columns[10:-1]

# Only the train gets processed, the rest will be processed on a rolling window in the testing phase
train = process(train, columns2norm)

In [ ]:
cols = [
    "skew1",
    "skew2",
    "iv",
    "RSI",
    "mktrf_beta_eom",
    "smb_beta_eom",
    "mom_beta_eom",
    "hml_beta_eom",
    "yield_dif_10y_1y_beta_eom",
] 
fig, axs = plt.subplots(3, 3)
fig.set_size_inches(18.5, 12.5, forward=True)
fig.suptitle("Distribution of Variables", fontsize=18)

for i, el in enumerate(cols):
    a = train.hist(el, ax=axs.flatten()[i], bins=100)

# Models

## Random Forest

In [ ]:
def rolling_forest(data, predictors, target, window, params):
    # First we form the window
    start_window = data.date.min()
    end_window = data.date.min() + BDay(window)
    pred_date = end_window + BDay(1)

    # Auxiliary placeholders and reg. constant
    data = data.sort_values(by=["date", "permno"])
    predictions_df = pd.DataFrame(
        index=[pred_date], columns=np.sort(data.permno.unique())
    )
    scores = []
    missing_dates = []

    # The train / predict loop
    while pred_date <= data.date.max():
        # The loop will go through business days, not necessarily trading days (hence the try clause).
        try:
            # Picking the window of data to train and instantiating the predictor model
            mask = (data["date"] >= start_window) & (data["date"] <= end_window)
            train_window = data[mask].set_index("permno")
            linreg = RandomForestRegressor(**params)
            # Fitting the model on the training window
            fit_ = linreg.fit(train_window[list(predictors)], train_window[target])
            # Predicting for each permno on the next day after fit
            predi_df = (
                data[data["date"] == pred_date]
                .loc[:, list(predictors) + ["permno"]]
                .set_index("permno")
            )
            predi = fit_.predict(predi_df)
            # Storing in a useful format for backtest
            one_predi = (
                pd.DataFrame(data=predi, index=predi_df.index, columns=[pred_date])
            ).T
            predictions_df = pd.concat(
                [predictions_df, one_predi], axis=0, ignore_index=False
            )
            # R^2 to help visualize training performance
            score_r2 = linreg.score(
                predi_df[list(predictors)], data[data["date"] == pred_date][target]
            )
            scores.append(score_r2)
        except:
            missing_dates.append(pred_date)
        # Update window position
        start_window = start_window + BDay(1)
        end_window = start_window + BDay(window)
        pred_date = end_window + BDay(1)
    # Real returns we will need
    real_returns = data.pivot(index="date", columns="permno", values="fret1d")

    return real_returns, predictions_df[1:], missing_dates, scores  # , r2_average

In [ ]:
params = {
    "n_estimators": [10, 20, 30],
    "max_features": ["sqrt", "auto", "log2"],
    "max_depth": [3, 4, 5],
    "min_samples_leaf": [1, 2, 3, 4],
    "min_samples_split": [2, 3, 4, 5],
}

train.insert(10, "constant", 1.0)
regressors = train.columns[10:-1]
target = ["fret1d"]

forest = RandomForestRegressor()

folds = 3
param_comb = 5

random_search = RandomizedSearchCV(
    forest,
    param_distributions=params,
    n_iter=param_comb,
    scoring="r2",
    n_jobs=4,
    cv=KFold(5),
    verbose=3,
    random_state=1001,
)
random_search.fit(train[regressors], train[target])

In [ ]:
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)

In [ ]:
params_forest = {
    "n_estimators": 30,
    "max_features": "sqrt",
    "max_depth": 5,
    "min_samples_leaf": 3,
    "min_samples_split": 4,
    "random_state": 1001,
}

regressors = train.columns[10:-1]
target = ["fret1d"]

(
    real_returns_forest,
    expected_returns_forest,
    missing_dates_forest,
    scores_forest,
) = rolling_forest(
    data=train, predictors=regressors, target=target, window=126, params=params_forest
)

## Lasso

We will tune the alpha on the whole train set and then again apply a rolling approach to predict

In [ ]:
def lasso_regression(data, predictors, target, alpha):
    lasso = Lasso(alpha=alpha,normalize=True, max_iter=1e5).fit(data[predictors],data[target])
    predi = lasso.predict(data[predictors]) 
    score=lasso.score(data[predictors],data[target])
    n = np.sum(lasso.coef_!=0)
    mse=metrics.mean_squared_error(data[target].to_numpy(), predi)
    
    return mse, score,n

In [ ]:
alpha_lasso = [1e-16, 1e-8, 1e-4, 1e-2, 1, 10]
res = pd.DataFrame(index=alpha_lasso, columns=["MSE"])
for i in range(len(alpha_lasso)):
    (
        res.loc[alpha_lasso[i], "MSE"],
        res.loc[alpha_lasso[i], "Score"],
        res.loc[alpha_lasso[i], "No."],
    ) = lasso_regression(train, regressors, target, alpha_lasso[i])

In [ ]:
def rolling_lasso(data, predictors, target, window, alpha):

    # First we form the window
    start_window = data.date.min()
    end_window = data.date.min() + BDay(window)
    pred_date = end_window + BDay(1)

    # Auxiliary placeholders and reg. constant
    data = data.sort_values(by=["date", "permno"])
    predictions_df = pd.DataFrame(
        index=[pred_date], columns=np.sort(data.permno.unique())
    )
    scores = []
    missing_dates = []

    # The train / predict loop
    while pred_date <= data.date.max():
        # The loop will go through business days, not necessarily trading days (hence the try clause).
        try:
            # Picking the window of data to train and instantiating the predictor model
            mask = (data["date"] >= start_window) & (data["date"] <= end_window)
            train_window = data[mask].set_index("permno")
            linreg = Lasso(alpha=alpha, normalize=True, max_iter=1e5)
            # Fitting the model on the training window
            fit_ = linreg.fit(train_window[list(predictors)], train_window[target])
            # Predicting for each permno on the next day after fit
            predi_df = (
                data[data["date"] == pred_date]
                .loc[:, list(predictors) + ["permno"]]
                .set_index("permno")
            )
            predi = fit_.predict(predi_df)
            # Storing in a useful format for backtest
            one_predi = (
                pd.DataFrame(data=predi, index=predi_df.index, columns=[pred_date])
            ).T
            predictions_df = pd.concat(
                [predictions_df, one_predi], axis=0, ignore_index=False
            )
            # R^2 to help visualize training performance
            score_r2 = linreg.score(
                predi_df[list(predictors)], data[data["date"] == pred_date][target]
            )
            scores.append(score_r2)
        except:
            missing_dates.append(pred_date)
        # Update window position
        start_window = start_window + BDay(1)
        end_window = start_window + BDay(window)
        pred_date = end_window + BDay(1)
    # Real returns we will need
    real_returns = data.pivot(index="date", columns="permno", values="fret1d")

    return real_returns, predictions_df[1:], missing_dates, scores  # , r2_average

In [ ]:
(
    real_returns_lasso,
    expected_returns_lasso,
    missing_dates_lasso,
    scores_lasso,
) = rolling_lasso(
    data=train, predictors=regressors, target=target, window=126, alpha=1e-8
)

In [ ]:
np.mean(scores_lasso)

In [ ]:
expected_returns_forest.to_hdf(
    hdf_path, key="Predictions", mode="a", data_columns=True, complib="zlib"
)
real_returns_forest.to_hdf(
    hdf_path, key="Real", mode="a", data_columns=True, complib="zlib"
)

In [ ]:
real_returns = pd.read_hdf(hdf_path, key="Real")
expected_returns = pd.read_hdf(hdf_path, key="Predictions")

# Optimization

In [ ]:
def prepare_returns(real, expected, days_to_test,years_lookback):
    real = real.loc[np.logical_and(real.index >= days_to_test - pd.Timedelta(days=252*years_lookback + 1), real.index < days_to_test), :]
    expected = expected.loc[np.logical_and(expected.index >= days_to_test - pd.Timedelta(days=252*years_lookback + 1), expected.index < days_to_test), :]   
    real = real.fillna(0.0)
    expected = expected.fillna(0.0)
    
    return real, expected

In [ ]:
real,ret = prepare_returns(real_returns,expected_returns,pd.datetime(2012, 12, 28),5)

In [ ]:
def MVP(wvec, *args):
     cov = args[0]
     var = wvec@cov@wvec
     return var

In [ ]:
def benchmark_drawdown_rolling(df, ret, d, lookback):
    d = pd.Timestamp(d)
    start = d - BDay(lookback)

    means = ret.loc[start : (d - BDay(1)), :].values
    means_df = ret.loc[start : (d - BDay(1)), :]

    weights_bench = df[(df["date"] >= start) & (df["date"] < d)]
    weights_bench = weights_bench[["date", "permno", "weightage_pct"]]
    weights_bench = weights_bench.pivot(
        index="date", columns="permno", values="weightage_pct"
    )
    weights_bench = weights_bench.loc[weights_bench.index.isin(means_df.index)]
    monthly_benchmark_returns = (means * weights_bench).sum(axis=1)
    monthly_benchmark_returns = pd.DataFrame(
        monthly_benchmark_returns, columns=["Benchmark_Return"]
    )

    # cumulative return
    monthly_benchmark_returns["Cum_ret"] = (
        1 + monthly_benchmark_returns["Benchmark_Return"]
    ).cumprod()

    # cumulative peaks
    monthly_benchmark_returns["Peaks"] = monthly_benchmark_returns["Cum_ret"].cummax()
    
    # drawdown from trailing peak
    monthly_benchmark_returns["Drawdown"] = (
        monthly_benchmark_returns["Cum_ret"] - monthly_benchmark_returns["Peaks"]
    ) / monthly_benchmark_returns["Peaks"]

    return monthly_benchmark_returns, monthly_benchmark_returns["Drawdown"].min()

In [ ]:
def configure_inputs_for_optimisation(df, ret, expret, d, limit, lookback):

    cov = ret.cov().values
    mu = expret.iloc[-1].values
    mu = mu - np.mean(mu)
    mu = winsorize(mu, limits=0.1).data

    if np.sum(mu > 0) == 0:
        mu -= mu.mean()

    # Bounds on Weights Based on Benchmark on day d
    bounds = [
        [] for _ in range(len(ret.columns))
    ]  # Empty placeholder for the 50 lower and upper bounds per day
    for en, i in enumerate(ret.columns, start=0):
        try:
            lower_bound = 0.9 * float(
                df.loc[df["date"] == d][df["permno"] == i].weightage_pct.values
            )
            higher_bound = 1.1 * float(
                df.loc[df["date"] == d][df["permno"] == i].weightage_pct.values
            )
            bounds[en].extend((lower_bound, higher_bound))
        except:
            lower_bound = 0.0
            higher_bound = 0.0
            bounds[en].extend((lower_bound, higher_bound))

    # Constraints on Factor Exposure
    cons = []
    fact = pd.DataFrame(columns=list(regressors) + ["permno"])
    fact.permno = ret.columns.tolist()
    fact.set_index("permno", inplace=True)
    fact = fact.fillna(0.0)

    # Exposure of strategy
    for stock in fact.index:
        for fct in fact.columns:
            flt = np.logical_and(df["date"] == d, df["permno"] == stock)
            if flt.any():
                fact.loc[stock, fct] = df.loc[flt, fct].values[0]

    B = fact.T[1:]

    # Benchmark Weights
    weights_Benchmark = df.loc[df["date"] == d][["permno", "weightage_pct"]]
    weights_Benchmark = weights_Benchmark.sort_values("permno")
    weights_Benchmark = weights_Benchmark.set_index(["permno"]).squeeze()
    for i in df.permno.unique():
        if i not in weights_Benchmark.index:
            weights_Benchmark[i] = 0.0
    weights_Benchmark = weights_Benchmark.sort_index()

    cons.append(
        {"type": "ineq", "fun": lambda wvec: B @ weights_Benchmark - B @ wvec + limit}
    )
    cons.append(
        {"type": "ineq", "fun": lambda wvec: B @ wvec - B @ weights_Benchmark + limit}
    )

    # Constraints on Weight Sum (long portfolio)
    cons.append({"type": "eq", "fun": lambda wvec: wvec.sum() - 1})

    # Constraints on Drawdown
    if (pd.Timestamp(d) - BDay(lookback)) in ret.index:
        _, benchmark_dd = benchmark_drawdown_rolling(df, ret, d, 21)
        cons.append(
            {"type": "ineq", "fun": lambda wvec: mu.T @ wvec - (benchmark_dd - 0.01)}
        )

    return cov, mu, cons, bounds

# Backtest

## Backtest on In-Sample Period 

In [ ]:
dtall = pd.DataFrame(index=ret.index, data=ret.index, columns=["dates"])  #
dt_m = dtall.groupby(
    pd.Grouper(freq="B")
).last()  # We can rebalane daily for final solution should be better results
dates_rebal = dt_m.index.values  # But less realistic (daily trading)

# We need to also exclude from rebalance dates the business days that were actually not trading days
dates_rebal = [x for x in dates_rebal if x not in missing_dates_forest]

In [ ]:
names = ret.columns
ptf = {}

# Initiate a dictionary of placeholders for the 3 strategies
ptf["msr_w_cons"] = pd.DataFrame(data=0.0, index=dates_rebal, columns=names)
ptf["minvar_w_cons"] = pd.DataFrame(data=0.0, index=dates_rebal, columns=names)

In [ ]:
# Optimization routine for the minimization problem
st_point_mvp = None

for d in tqdm(dates_rebal):
    print(d)
    start_time = time.time()

    ret, expret = prepare_returns(real_returns, expected_returns, d, 3)

    # we prepare inputs:
    cov, mu, cons, bounds = configure_inputs_for_optimisation(
        train, ret, expret, d, 0.05, 21
    )

    # define starting point
    if st_point_msr is None:
        st_point_msr = np.ones(ret.columns.size) / ret.columns.size
    if st_point_mvp is None:
        st_point_mvp = np.ones(ret.columns.size) / ret.columns.size

    res = minimize(
        MVP,
        st_point_mvp,
        args=(cov, mu),
        constraints=cons,
        bounds=bounds,
        method="SLSQP",
        options={"ftol": 1e-16, "disp": False},
    )
    ptf["minvar_w_cons"].loc[d] = res.x
    # Set next starting point
    st_point_mvp = res.x

In [ ]:
def clc_str_return(w, ret, str_name):
    w_i = np.zeros(len(ret.columns.tolist()))
    ret_str = pd.DataFrame(index = ret.index, columns=[str_name])
    for i in ret.index:
        ret_str.loc[i, str_name] = w_i@ret.loc[i,:]
        if i in w.index:
            w_i = w.loc[i,:]
    return ret_str

In [ ]:
ret_str = clc_str_return(ptf["msr_w_cons"], ret, "msr_w_cons")
ret_str = ret_str.join(clc_str_return(ptf["minvar_w_cons"], ret, "minvar_w_cons"))

## Backtest on Out-of-Sample Period 

The function is only different so that it also normalizes the rolling window we are fitting/testing on. This is important so we avoid lookahead bias in our test set.

In [ ]:
def rolling_forest_test(data, predictors, target, window, params):
    # First we form the window
    start_window = data.date.min()
    end_window = data.date.min() + BDay(window)
    pred_date = end_window + BDay(1)

    # Auxiliary placeholders and reg. constant
    data = data.sort_values(by=["date", "permno"])
    predictions_df = pd.DataFrame(
        index=[pred_date], columns=np.sort(data.permno.unique())
    )
    scores = []
    missing_dates = []

    # The train / predict loop
    while pred_date <= data.date.max():
        # The loop will go through business days, not necessarily trading days (hence the try clause).
        try:
            # Picking the window of data to train and instantiating the predictor model
            mask = (data["date"] >= start_window) & (data["date"] <= end_window)
            train_window = data[mask].set_index("permno")
            train_window = process(train_window, list(predictors)[1:])
            linreg = RandomForestRegressor(**params)
            # Fitting the model on the training window
            fit_ = linreg.fit(train_window[list(predictors)], train_window[target])
            # Predicting for each permno on the next day after fit
            predi_df = (
                data[data["date"] == pred_date]
                .loc[:, list(predictors) + ["permno"]]
                .set_index("permno")
            )
            predi = fit_.predict(predi_df)
            # Storing in a useful format for backtest
            one_predi = (
                pd.DataFrame(data=predi, index=predi_df.index, columns=[pred_date])
            ).T
            predictions_df = pd.concat(
                [predictions_df, one_predi], axis=0, ignore_index=False
            )
            # R^2 to help visualize training performance
            score_r2 = linreg.score(
                predi_df[list(predictors)], data[data["date"] == pred_date][target]
            )
            scores.append(score_r2)
        except:
            missing_dates.append(pred_date)
        # Update window position
        start_window = start_window + BDay(1)
        end_window = start_window + BDay(window)
        pred_date = end_window + BDay(1)
    # Real returns we will need
    real_returns = data.pivot(index="date", columns="permno", values="fret1d")

    return real_returns, predictions_df[1:], missing_dates, scores  # , r2_average

In [ ]:
params_forest = {
    "n_estimators": 30,
    "max_features": "sqrt",
    "max_depth": 5,
    "min_samples_leaf": 3,
    "min_samples_split": 4,
    "random_state": 1001,
}

regressors = test.columns[10:-1]
target = ["fret1d"]

(
    real_returns_forest_test,
    expected_returns_forest_test,
    missing_dates_forest_test,
    scores_forest_test,
) = rolling_forest_test(
    data=test, predictors=regressors, target=target, window=21, params=params_forest
)
real_returns_forest_test = real_returns_forest_test[
    real_returns_forest_test.index >= expected_returns_forest_test.index.min()
]

In [ ]:
ret, expret = prepare_returns(real_returns_forest_test,expected_returns_forest_test,pd.datetime(2019, 12, 31),9)

In [ ]:
dtall = pd.DataFrame(index=ret.index, data=ret.index, columns=["dates"])  #
dt_m = dtall.groupby(
    pd.Grouper(freq="B")
).last()  # We can rebalane daily for final solution should be better results
dates_rebal = dt_m.index.values  # But less realistic (daily trading)

# We need to also exclude from rebalance dates the business days that were actually not trading days
# #missing_dates = [i.to_datetime64() for i in missing_dates] # Uncomment if needed a convert!
dates_rebal = [x for x in dates_rebal if x not in missing_dates_forest_test]

# dates_ = [i.date() for i in dates_rebal]
real_returns_forest_test = real_returns_forest_test.loc[
    ~real_returns_forest_test.index.isin(missing_dates_forest_test)
]
expected_returns_forest_test = expected_returns_forest_test.loc[
    ~expected_returns_forest_test.index.isin(missing_dates_forest_test)
]

In [ ]:
names = ret.columns
ptf = {}

# Initiate a dictionary of placeholders for the 3 strategies
ptf["msr_w_cons"] = pd.DataFrame(data=0.0, index=dates_rebal, columns=names)
ptf["minvar_w_cons"] = pd.DataFrame(data=0.0, index=dates_rebal, columns=names)

In [ ]:
bnch_ret = (
    df_betas.groupby(["date"]).apply(weighted_avg, "ret", "weightage_pct") + 1
).cumprod()

bnch_ret = bnch_ret.loc[
    (bnch_ret.index >= ret_str.index.min()) & (bnch_ret.index <= ret_str.index.max())
]

In [ ]:
# Optimization rutine for the minimization problem
st_point_mvp = None

for d in tqdm(dates_rebal):
    ret, expret = prepare_returns(
        real_returns_forest_test, expected_returns_forest_test, d, 3
    )

    # we prepare inputs:
    cov, mu, cons, bounds = configure_inputs_for_optimisation(
        test, ret, expret, d, 0.05, 21
    )

    # define starting point
    if st_point_msr is None:
        st_point_msr = np.ones(ret.columns.size) / ret.columns.size
    if st_point_mvp is None:
        st_point_mvp = np.ones(ret.columns.size) / ret.columns.size

    res = minimize(
        MVP,
        st_point_mvp,
        args=(cov, mu),
        constraints=cons,
        bounds=bounds,
        method="SLSQP",
        options={"ftol": 1e-16, "disp": False},
    )

    ptf["minvar_w_cons"].loc[d] = res.x

    # Set next starting point
    st_point_mvp = res.x


In [ ]:
ret, expret = prepare_returns(
    real_returns_forest_test, expected_returns_forest_test, pd.datetime(2019, 12, 31), 9
)
ret_str = clc_str_return(ptf["msr_w_cons"], ret, "msr_w_cons")
ret_str = ret_str.join(clc_str_return(ptf["minvar_w_cons"], ret, "minvar_w_cons"))


In [ ]:
qs.reports.full(ret_str["minvar_w_cons"], bnch_ret)